# NavWareSet Quantitative Analysis

This notebook is a quantitative analysis of 545 hand selected robot-human tracks from the NavWareSet dataset. Almost all tracks from all scenarios where compliant and non-compliant behavior are present are analysed. All metrics were calculated for each possible combination of scenario, robot and behavior. Our objective is not to propose new metrics, but to test how our new dataset scores against metrics already proposed in the literature about robot social navigation. 

The chosen metrics were:
- Vavg (m/s) [1] - Average velocity of the robot across a number of tracks;
- Aavg (m/s2) [1] - Average aceleration of the robot across a number of tracks;
- Javg (m/s3) [1] - Average Jerk of the robot across a number of tracks;
- PLavg (m) [1] - Agerage Path Length across a number of tracks;
- PSC (%) [2] - Ratio of the trajectory with the minimum distance to a human under a given threshold. If the threshold is 0.5m, it is referred to as Personal Space Compliance (PSC);
- DHmin [3] - Average minimum distance to a human across a number of tracks;

[(1) - Arena-Bench: A Benchmarking Suite for Obstacle Avoidance Approaches in Highly Dynamic Environments](https://arxiv.org/pdf/2206.05728)

[(2) - iGibson 2.0: Object-Centric Simulation for Robot Learning of Everyday Household Tasks](https://arxiv.org/pdf/2108.03272)

[(3) - Principles and Guidelines for Evaluating Social Robot Navigation Algorithms](https://arxiv.org/pdf/2306.16740)

## Separatring the scenes

In total, 5 scenes were analysed:
- Frontal Approach (fa);
- Pedestrian Obstruction (po);
- Blind Corner (bc);
- Perpendicular Crossing (pc);
- Circular Crossing (cc).

Two robots were used:
- Cleapath Jackal (j);
- Toyota HSR (h).

And two robot behaviors were observed:
- Compliant (c);
- Non-compliant (n).

In the cell below all scenes are divided acording to the classifications above.

In [ ]:
fa_h_c = [1, 27]
fa_h_n = [8, 34]
fa_j_c = [15, 41]
fa_j_n = [21, 47]
po_h_c = [2, 28]
po_h_n = [9, 35]
po_j_c = [16, 42]
po_j_n = [22]
bc_h_c = [3, 29]
bc_h_n = [10, 36]
bc_j_c = []
bc_j_n = []
pc_h_c = []
pc_h_n = []
pc_j_c = []
pc_j_n = []
cc_h_c = []
cc_h_n = []
cc_j_c = []
cc_j_n = []